In [2]:
# ===============================
# 🧠 Quantum Phase Estimation (QPE)
# with Student Practice Tasks (Error-Free)
# ===============================

!pip install qiskit qiskit-aer matplotlib pylatexenc --quiet

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Safe display function (won't crash if pylatexenc is missing)
# -------------------------------
def safe_draw(qc):
    """Safely draw the circuit using matplotlib if available."""
    try:
        display(qc.draw("mpl"))
    except Exception as e:
        print("(⚠️ Circuit visualization skipped — optional library missing)")
        print("Error:", e)
        print(qc.draw(output='text'))

# -------------------------------
# Inverse Quantum Fourier Transform
# -------------------------------
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for j in reversed(range(n)):
        qc.h(j)
        for k in reversed(range(j)):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

# -------------------------------
# QPE Circuit
# -------------------------------
def qpe_circuit(num_count_qubits, unitary, theta):
    """Construct the Quantum Phase Estimation (QPE) circuit."""
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate |1⟩
    qc.x(num_count_qubits)

    # Step 3: Apply controlled-U^2^j operations (simulated as controlled phase)
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Apply inverse QFT
    inverse_qft(qc, num_count_qubits)

    # Step 5: Measure
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

# -------------------------------
# Run QPE Simulation
# -------------------------------
def run_qpe(num_count_qubits=3, theta=0.125):
    simulator = AerSimulator()
    qc = qpe_circuit(num_count_qubits, "U", theta)

    # Visualization before measurement (Inverse QFT structure)
    print("\n🔹 QPE Circuit with Inverse QFT Visualization:")
    safe_draw(qc)

    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()

    # Theoretical expected output
    expected_decimal = int(theta * (2 ** num_count_qubits))
    expected_binary = format(expected_decimal, f'0{num_count_qubits}b')
    print(f"Expected binary output ≈ {expected_binary} (for θ={theta})")

    # Display histogram
    plot_histogram(counts, title=f"QPE Result for θ = {theta}")
    plt.show()

# -------------------------------
# Run QPE with Noise Simulation
# -------------------------------
def run_qpe_with_noise(num_count_qubits=3, theta=0.125):
    """Run QPE under a depolarizing noise model."""
    simulator = AerSimulator()

    # Define a simple depolarizing noise model
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(depolarizing_error(0.02, 2), ['cp', 'cx'])

    qc = qpe_circuit(num_count_qubits, "U", theta)
    print("\n🔹 QPE Circuit (with Noise):")
    safe_draw(qc)

    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048, noise_model=noise_model).result()
    counts = result.get_counts()

    plot_histogram(counts, title=f"QPE with Noise (θ={theta})")
    plt.show()

# -------------------------------
# 🔹 PRACTICE TASKS SECTION
# -------------------------------

print("🔸 Task 1: Changing Phase Values (θ = 0.125, 0.25, 0.375, 0.5)")
for theta in [0.125, 0.25, 0.375, 0.5]:
    print(f"\nRunning QPE for θ = {theta}")
    run_qpe(num_count_qubits=3, theta=theta)

# --------------------------------
print("\n🔸 Task 2: Increasing the Number of Counting Qubits (4 qubits)")
run_qpe(num_count_qubits=4, theta=0.125)

# --------------------------------
print("\n🔸 Task 3: Comparison with Theoretical Output")
run_qpe(num_count_qubits=3, theta=0.375)

# --------------------------------
print("\n🔸 Task 4: Visualizing Inverse QFT (already shown above)")

# --------------------------------
print("\n🔸 Task 5: Running with Noise Simulation")
run_qpe_with_noise(num_count_qubits=3, theta=0.125)

print("\n✅ All Student Practice Tasks Completed Successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
🔸 Task 1: Changing Phase Values (θ = 0.125, 0.25, 0.375, 0.5)

Running QPE for θ = 0.125

🔹 QPE Circuit with Inverse QFT Visualization:
(⚠️ Circuit visualization skipped — optional library missing)
Error: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
     ┌───┐                                                                 »
q_0: ┤ H ├─■───────────────────────────────────────■──────────────■────────»
     ├───┤ │                                       │        ┌───┐ │P(-π/2) »
q_1: ┤ H ├─┼────────■────────────────────■─────────┼────────┤ H ├─■────────»
     ├───┤ │        │              ┌───┐ │P(-π/2)  │P(-π/4) └───┘          »
q_2: ┤ H ├─┼────────┼────────■─────┤ H ├─■─────────■───────────────────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π) └───┘                                   »
q_3: ┤ X ├─■──────